In [117]:
import torch
import os
import math


import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import gen_batches
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from copy import deepcopy
from time import time
import numpy as np
from PIL import Image


# just to ignore the matplotlib warnings to make the plots look better
import warnings
warnings.filterwarnings('ignore')
# test whether the cuda is available
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1
True


In [118]:
import pandas as pd
train_biz=pd.read_csv('./train_photo_to_biz_ids.csv')
test_biz=pd.read_csv('./test_photo_to_biz.csv')
train=pd.read_csv('./train.csv')
sub=pd.read_csv('./sample_submission.csv')

In [119]:
#merging
data=pd.merge(train_biz,train, on='business_id',how='left')
data=data.dropna(subset=['labels'])

data_test=pd.merge(test_biz,sub, on='business_id',how='left') 

In [120]:
# convert string to list of int
labels=data['labels'].tolist()

for index,x in enumerate(labels):
     labels[index]= [int(c) for c in str(x).split(' ')]
data['labels'] = labels

In [121]:
len(data_train)

234545

In [122]:
# # split data
# train_size = int(len(data)*0.8)
# val_size = len(data)-train_size
# train_data, val_data = random_split(data_train, lengths=[train_size,val_size])

In [123]:
from sklearn import metrics, model_selection, preprocessing
df_train, df_valid = model_selection.train_test_split(data, test_size=0.1, random_state=40)

In [135]:
from torch.utils.data.dataset import Dataset
class NusDataset(Dataset):
#     def __init__(self, data_path, data, transforms):
    def __init__(self, data_path, data):
#         self.transforms = transforms
   
        self.image_id = data['photo_id'].tolist()
        self.labels=data['labels'].tolist()
        
        self.classes = [0,1,2,3,4,5,6,7,8]
        self.data_path = data_path
          
        for index in range(len(self.labels)):
            labels = self.labels[index]
            vector=[0,0,0,0,0,0,0,0,0]
            for i in range(9):
                if i in labels:
                    vector[i] = 1
            self.labels[index] = np.array(vector, dtype=float)

    def __getitem__(self, index):
        label = self.labels[index]
        img_path = os.path.join(self.data_path, str(self.image_id[index])+'.jpg')
        img = Image.open(img_path)
#         if self.transforms is not None:
#             img = self.transforms(img)
        return img, label

        

    def __len__(self):
        return len(self.imgs)

In [136]:
dataset_val = NusDataset('./train_photos', df_valid)
dataset_train = NusDataset('./train_photos', df_train)
df_train

,photo_id,business_id,labels
184802,125301,1323,"[0, 2, 3, 5]"
152400,267471,2379,"[1, 2, 3, 4, 5, 6]"
174145,321518,1031,"[6, 8]"
152138,356872,487,"[1, 2, 3, 4, 5, 6]"
39775,108147,998,"[1, 2, 4, 5, 6, 7]"
...,...,...,...
93231,345331,3883,"[2, 5, 6, 8]"
211366,315347,3696,"[1, 2, 3, 4, 5, 6]"
30741,259647,3339,"[1, 2, 3, 4, 5, 6, 7]"
112934,269721,1931,"[2, 6]"


In [140]:
dataset_train[1][1]

array([0., 1., 1., 1., 1., 1., 1., 0., 0.])